# findling around with blocks

This is my test ground to get to know the Blocks framework for deep learning and make keep some basics examples.

In [8]:
#MNIST example of blocks : https://blocks.readthedocs.org/en/latest/tutorial.html
from theano import tensor
x = tensor.matrix('features')

from blocks.bricks import Linear, Rectifier, Softmax
input_to_hidden = Linear(name='input_to_hidden', input_dim=784, output_dim=100)
h = Rectifier().apply(input_to_hidden.apply(x))
hidden_to_output = Linear(name='hidden_to_output', input_dim=100, output_dim=10)
y_hat = Softmax().apply(hidden_to_output.apply(h))

y = tensor.lmatrix('targets')
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError
cost = CategoricalCrossEntropy().apply(y.flatten(), y_hat)


from blocks.bricks import WEIGHT
from blocks.graph import ComputationGraph
from blocks.filter import VariableFilter
cg = ComputationGraph(cost)
W1, W2 = VariableFilter(roles=[WEIGHT])(cg.variables)
cost = cost + 0.005 * (W1 ** 2).sum() + 0.005 * (W2 ** 2).sum()
cost.name = 'cost_with_regularization'


#Short for above
from blocks.bricks import MLP
x = tensor.matrix('features')
mlp = MLP(activations=[Rectifier(), Softmax()], dims=[784, 400, 10]).apply(x)

from blocks.initialization import IsotropicGaussian, Constant
input_to_hidden.weights_init = hidden_to_output.weights_init = IsotropicGaussian(0.01)
input_to_hidden.biases_init = hidden_to_output.biases_init = Constant(0)
input_to_hidden.initialize()
hidden_to_output.initialize()

W1.get_value() 

from fuel.datasets import MNIST
mnist = MNIST(("train",))


from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
data_stream = Flatten(DataStream.default_stream(
    mnist,
    iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=256)))

from blocks.algorithms import GradientDescent, Scale
algorithm = GradientDescent(cost=cost, parameters=cg.parameters,
                             step_rule=Scale(learning_rate=0.1))



mnist_test = MNIST(("test",))
data_stream_test = Flatten(DataStream.default_stream(
    mnist_test,
    iteration_scheme=SequentialScheme(
       mnist_test.num_examples, batch_size=64)))



from blocks.extensions.monitoring import DataStreamMonitoring,DataStreamMonitoring
classification_error= MisclassificationRate().apply(np.argmax(y_hat),y)
monitor = DataStreamMonitoring(
    variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
main_loop = MainLoop(data_stream=data_stream, algorithm=algorithm,
                     extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
main_loop.run() 


-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 test_cost_with_regularization: 2.34214115531
	 test_misclassificationrate_apply_error_rate: 1.0


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: False
	 epochs_done: 1
	 iterations_done: 235
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the iteration 235:
	 test_cost_with_regulariz

In [8]:
#I started from the example above and added some couple of things


import numpy as np
#Importing dataset
from fuel.datasets import MNIST
mnist = MNIST(("train",))

#Data Stream
from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten
data_stream = Flatten(DataStream.default_stream(
    mnist,
    iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=256)))

#Model
from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor

x = tensor.matrix('features')
y1 = tensor.lmatrix(name='targets')

#mlp = MLP(activations=[Logistic(name='sigmoid_0'),
#          Rectifier(name='rect1'),Softmax(name='softmax_0')], dims=[784, 300, 200,10],
#          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))

mlp = MLP(activations=[Rectifier(name='rect0'),Logistic(name='sigmoid_1'),Softmax(name='softmax_0')], dims=[784, 300, 300, 10],
          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))


y = mlp.apply(x)
print(y)
mlp.push_initialization_config()
mlp.children[0].weights_init = Constant(0.01)
mlp.initialize()
#print(mlp.children[0].parameters[0].get_value()) 


#Cost
from blocks.graph import ComputationGraph
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError,MisclassificationRate
from blocks.algorithms import Scale,GradientDescent
cost = CategoricalCrossEntropy().apply(y1.flatten(),y)
algorithm = GradientDescent(cost=cost, parameters=ComputationGraph(cost).parameters,
                             step_rule=Scale(learning_rate=0.005))


#data stream test
mnist_test = MNIST(("test",))
data_stream_test = Flatten(DataStream.default_stream(
    mnist_test,
    iteration_scheme=SequentialScheme(
       mnist_test.num_examples, batch_size=64)))


#Monitor
from blocks.extensions.monitoring import DataStreamMonitoring
classification_error= MisclassificationRate().apply(tensor.argmax(y1),y)
monitor = DataStreamMonitoring(
    variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

#MainLoop
from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing
main_loop = MainLoop(algorithm,data_stream=data_stream, model=mlp,
                     extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
main_loop.run()

#%debug

mlp_apply_output

-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteration 0:
	 test_categoricalcrossentropy_apply_cost: 10.2292599936
	 test_misclassificationrate_apply_error_rate: 0.936305732484


-------------------------------------------------------------------------------
AFTER ANOTHER EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: False
	 epochs_done: 1
	 iterations_done: 235
	 received_first_batch: True
	 resumed_from: None
	 training_started: True
Log records from the ite

# Results

batch_size, activation, dims, cost_used, learning_rate, epoch,  cost_with_regularization, misclassificationrate

64,[Logistic(), Rectifier(),Softmax()],[784, 300, 200,10], CategoricalCrossEntropy, 0.05, 4, 0.165451308803, 0.927249203822

...








In [7]:

from itertools import permutations,product
from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor

#Trying all 3-permutations of logistic and rectifier with the same initial configuration of every other parameters. 
for m,j,h in product([Logistic, Rectifier],[Logistic, Rectifier],[Logistic, Rectifier]):
    print(m,j,h)
    activation = [m(name=str(type(m))+'0'),j(name=str(type(j))+'1'),h(name=str(type(h))+'2'),Softmax(name='softmax_4')]
    
    mlp = MLP(activations=activation, dims=[784,500,300,150,10],
          weights_init=IsotropicGaussian(), biases_init=Constant(0.01))

    try_model_on_mnist(mlp)
    
    

(<class 'blocks.bricks.simple.Logistic'>, <class 'blocks.bricks.simple.Logistic'>, <class 'blocks.bricks.simple.Logistic'>)
[<blocks.bricks.simple.Logistic object at 0x7fd8cdcffbd0: name=<class 'blocks.bricks.interfaces.ActivationDocumentation'>0>, <blocks.bricks.simple.Logistic object at 0x7fd8cdcffb90: name=<class 'blocks.bricks.interfaces.ActivationDocumentation'>1>, <blocks.bricks.simple.Logistic object at 0x7fd8cdcffc50: name=<class 'blocks.bricks.interfaces.ActivationDocumentation'>2>, <blocks.bricks.simple.Softmax object at 0x7fd8cdcffcd0: name=softmax_4>]

-------------------------------------------------------------------------------
BEFORE FIRST EPOCH
-------------------------------------------------------------------------------
Training status:
	 batch_interrupt_received: False
	 epoch_interrupt_received: False
	 epoch_started: True
	 epochs_done: 0
	 iterations_done: 0
	 received_first_batch: False
	 resumed_from: None
	 training_started: True
Log records from the iteratio

In [1]:

print(list(permutations([Logistic, Rectifier],r=3)))
print(Logistic)
print(list(permutations([1,2,3],3)))

NameError: name 'permutations' is not defined

In [6]:
import numpy as np

#Importing dataset
from fuel.datasets import MNIST
mnist = MNIST(("train",))
mnist_test = MNIST(("test",))

from fuel.streams import DataStream
from fuel.schemes import SequentialScheme
from fuel.transformers import Flatten

from blocks.graph import ComputationGraph
from blocks.bricks.cost import CategoricalCrossEntropy,SquaredError,MisclassificationRate
from blocks.algorithms import Scale,GradientDescent

from blocks.bricks import Tanh,Linear, Rectifier, Softmax, MLP, Logistic
from blocks.initialization import IsotropicGaussian, Constant
from theano import tensor
from blocks.extensions.monitoring import DataStreamMonitoring

from blocks.main_loop import MainLoop
from blocks.extensions import FinishAfter, Printing


def try_model_on_mnist(model,batch_size=64,nb_epoch=5):
    #Data Stream
    data_stream = Flatten(DataStream.default_stream(
        mnist,
        iteration_scheme=SequentialScheme(mnist.num_examples, batch_size=batch_size)))

    #Theano variables initialization

    x = tensor.matrix('features')
    y1 = tensor.lmatrix(name='targets')
    y = model.apply(x)
    model.push_initialization_config()
    model.children[0].weights_init = Constant(0.01)
    model.initialize()

    
    #Cost
    cost = CategoricalCrossEntropy().apply(y1.flatten(),y)
    algorithm = GradientDescent(cost=cost, parameters=ComputationGraph(cost).parameters,
                                 step_rule=Scale(learning_rate=0.005))

    #data stream test
    data_stream_test = Flatten(DataStream.default_stream(
        mnist_test,
        iteration_scheme=SequentialScheme(
           mnist_test.num_examples, batch_size=batch_size)))
    
    #Monitor
    classification_error= MisclassificationRate().apply(np.argmax(y1),y)
    monitor = DataStreamMonitoring(
        variables=[cost,classification_error], data_stream=data_stream_test, prefix="test")

    #MainLoop
    main_loop = MainLoop(algorithm,data_stream=data_stream, model=mlp,
                         extensions=[monitor, FinishAfter(after_n_epochs=4), Printing()])
    main_loop.run()


Examples from internet as quick reminders

In [ ]:
# Example to cast numpy datasets to h5py from http://fuel.readthedocs.org/en/latest/h5py_dataset.html
import numpy
numpy.save(
    'train_vector_features.npy',
    numpy.random.normal(size=(90, 10)).astype('float32'))
numpy.save(
    'test_vector_features.npy',
    numpy.random.normal(size=(10, 10)).astype('float32'))
numpy.save(
    'train_image_features.npy',
numpy.random.randint(2, size=(90, 3, 5, 5)).astype('uint8'))
numpy.save(
    'test_image_features.npy',
    numpy.random.randint(2, size=(10, 3, 5, 5)).astype('uint8'))
numpy.save(
    'train_targets.npy',
    numpy.random.randint(10, size=(90, 1)).astype('uint8'))
numpy.save(
    'test_targets.npy',
    numpy.random.randint(10, size=(10, 1)).astype('uint8'))


train_vector_features = numpy.load('train_vector_features.npy')
test_vector_features = numpy.load('test_vector_features.npy')
train_image_features = numpy.load('train_image_features.npy')
test_image_features = numpy.load('test_image_features.npy')
train_targets = numpy.load('train_targets.npy')
test_targets = numpy.load('test_targets.npy')

import h5py
f = h5py.File('dataset.hdf5', mode='w')
vector_features = f.create_dataset(
    'vector_features', (100, 10), dtype='float32')
image_features = f.create_dataset(
    'image_features', (100, 3, 5, 5), dtype='uint8')
targets = f.create_dataset(
    'targets', (100, 1), dtype='uint8')

vector_features[...] = numpy.vstack(
    [train_vector_features, test_vector_features])
image_features[...] = numpy.vstack(
    [train_image_features, test_image_features])
targets[...] = numpy.vstack([train_targets, test_targets])


vector_features.dims[0].label = 'batch'
vector_features.dims[1].label = 'feature'
image_features.dims[0].label = 'batch'
image_features.dims[1].label = 'channel'
image_features.dims[2].label = 'height'
image_features.dims[3].label = 'width'
targets.dims[0].label = 'batch'
targets.dims[1].label = 'index'

split_array = numpy.empty(
    6,
    dtype=numpy.dtype([
        ('split', 'a', 5),
        ('source', 'a', 15),
        ('start', numpy.int64, 1),
        ('stop', numpy.int64, 1),
        ('indices', h5py.special_dtype(ref=h5py.Reference)),
        ('available', numpy.bool, 1),
        ('comment', 'a', 1)]))
split_array[0:3]['split'] = 'train'.encode('utf8')
split_array[3:6]['split'] = 'test'.encode('utf8')
split_array[0:6:3]['source'] = 'vector_features'.encode('utf8')
split_array[1:6:3]['source'] = 'image_features'.encode('utf8')
split_array[2:6:3]['source'] = 'targets'.encode('utf8')
split_array[0:3]['start'] = 0
split_array[0:3]['stop'] = 90
split_array[3:6]['start'] = 90
split_array[3:6]['stop'] = 100
split_array[:]['indices'] = h5py.Reference()
split_array[:]['available'] = True
split_array[:]['comment'] = '.'.encode('utf8')
f.attrs['split'] = split_array

from fuel.datasets.hdf5 import H5PYDataset
split_dict = {
    'train': {'vector_features': (0, 90), 'image_features': (0, 90),
              'targets': (0, 90)},
    'test': {'vector_features': (90, 100), 'image_features': (90, 100),
             'targets': (90, 100)}}
f.attrs['split'] = H5PYDataset.create_split_array(split_dict)

f.flush()
f.close()

#Manipulations

train_set = H5PYDataset('dataset.hdf5', which_sets=('train',))
print(train_set.num_examples)

test_set = H5PYDataset('dataset.hdf5', which_sets=('test',))
print(test_set.num_examples)

train_set = H5PYDataset(
    'dataset.hdf5', which_sets=('train',), subset=slice(0, 80))
print(train_set.num_examples)
valid_set = H5PYDataset(
    'dataset.hdf5', which_sets=('train',), subset=slice(80, 90))
print(valid_set.num_examples)

print(train_set.provides_sources)
print(train_set.axis_labels['image_features'])
print(train_set.axis_labels['vector_features'])
print(train_set.axis_labels['targets'])


handle = train_set.open()
data = train_set.get_data(handle, slice(0, 10))
print((data[0].shape, data[1].shape, data[2].shape))
train_set.close(handle)



In [ ]:
#This whole cell is copy-paste from https://ift6266h16.wordpress.com/class-project/getting-started/


from fuel.streams import DataStream
from fuel.schemes import ShuffledScheme
from fuel.transformers.image import RandomFixedSizeCrop
from fuel.transformers import Flatten

# Load the training set
train = DogsVsCats(('train',), subset=slice(0, 20000))

# We now create a "stream" over the dataset which will return shuffled batches
# of size 128. Using the DataStream.default_stream constructor will turn our
# 8-bit images into floating-point decimals in [0, 1].
stream = DataStream.default_stream(
    train,
    iteration_scheme=ShuffledScheme(train.num_examples, 128)
)

# Our images are of different sizes, so we'll use a Fuel transformer
# to take random crops of size (32 x 32) from each image
cropped_stream = RandomFixedSizeCrop(
    stream, (32, 32), which_sources=('image_features',))

# We'll use a simple MLP, so we need to flatten the images
# from (channel, width, height) to simply (features,)
flattened_stream = Flatten(
    cropped_stream, which_sources=('image_features',))


#----------------
# Create the Theano MLP
import theano
from theano import tensor
import numpy

X = tensor.matrix('image_features')
T = tensor.lmatrix('targets')

W = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(3072, 500)), 'W')
b = theano.shared(numpy.zeros(500))
V = theano.shared(
    numpy.random.uniform(low=-0.01, high=0.01, size=(500, 2)), 'V')
c = theano.shared(numpy.zeros(2))
params = [W, b, V, c]

H = tensor.nnet.sigmoid(tensor.dot(X, W) + b)
Y = tensor.nnet.softmax(tensor.dot(H, V) + c)

loss = tensor.nnet.categorical_crossentropy(Y, T.flatten()).mean()

# Use Blocks to train this network
from blocks.algorithms import GradientDescent, Scale
from blocks.extensions import Printing
from blocks.extensions.monitoring import TrainingDataMonitoring
from blocks.main_loop import MainLoop

algorithm = GradientDescent(cost=loss, parameters=params,
                            step_rule=Scale(learning_rate=0.1))

# We want to monitor the cost as we train
loss.name = 'loss'
extensions = [TrainingDataMonitoring([loss], every_n_batches=1),
              Printing(every_n_batches=1)]

main_loop = MainLoop(data_stream=flattened_stream, algorithm=algorithm,
                     extensions=extensions)
main_loop.run()


In [1]:
#part of RNN tutorial: http://blocks.readthedocs.org/en/latest/rnn.html

import numpy
import theano
from theano import tensor
from blocks import initialization
from blocks.bricks import Identity
from blocks.bricks.recurrent import SimpleRecurrent
x = tensor.tensor3('x')
rnn = SimpleRecurrent(
    dim=3, activation=Identity(), weights_init=initialization.Identity())
rnn.initialize()
h = rnn.apply(x)
f = theano.function([x], h)
print(f(numpy.ones((3, 1, 3), dtype=theano.config.floatX))) 

[[[ 1.  1.  1.]]

 [[ 2.  2.  2.]]

 [[ 3.  3.  3.]]]


In [ ]:

from blocks.algorithms.StepRule import StepRule
#http://blocks.readthedocs.org/en/latest/api/algorithms.html?highlight=scale#blocks.algorithms.StepRule
class MyOwnStepRule(StepRule)
    def __init__(self):
        pass
    
    def compute_step(parameter,previous_step):
        pass
    
    def compute_steps(previous_steps):
        pass